In [102]:
import openai 
import pandas as pd 
import os 
from dotenv import load_dotenv

load_dotenv()

df=pd.read_excel('training_data_set.xlsx')
df

,Prompt,Response
0,What is the candidate name?,"His full name is Tran Cong Nam Anh, but he is ..."
1,Tell me fun fact about Louis?,Louis obtained a degree in Finance and Banking...
2,Which university does Louis or Tran Cong Nam A...,He/Louis is a third-year student majoring in C...
3,How many programming languages has Louis emplo...,"He has utilized Java, Python, Ruby, JavaScript..."
4,Please provide a list of the software developm...,Project Name: Missing Network Web Application ...
5,What were the job responsibilities that Louis ...,"In his previous internship, Louis held the rol..."
6,What is his current CGPA as well as his academ...,"Louis's CGPA is 4.8/5, earning him the honor o..."
7,What sports can Louis play ?,Louis is proficient in both badminton and foot...
8,When is his birthday ?,"Louis was born in 7th January, 1998. He is cur..."
9,What is his favorite football club and his fa...,Louis is the huge fan of Liverpool football cl...


In [103]:
system_prompt = "You are an assistant that answers all questions related to the candidate Tran Cong Nam Anh, referred to as his full name, or Louis Anh Tran, as his short name. The questions can revolve around his education, background, working experiences, hobbies, leadership experiences, academic achievements, and so on."


In [104]:
all_conversations=list()

for idx,row in df.iterrows():
    list_of_roles=[{"role":"system","content":system_prompt},{"role":"user","content":row['Prompt']},{"role":"assistant","content":row['Response']}]
    all_conversations.append({"messages":list_of_roles})

print(all_conversations)

[{'messages': [{'role': 'system', 'content': 'You are an assistant that answers all questions related to the candidate Tran Cong Nam Anh, referred to as his full name, or Louis Anh Tran, as his short name. The questions can revolve around his education, background, working experiences, hobbies, leadership experiences, academic achievements, and so on.'}, {'role': 'user', 'content': 'What is the candidate name?'}, {'role': 'assistant', 'content': 'His full name is Tran Cong Nam Anh, but he is commonly referred to as Louis. He is from Ho Chi Minh city, Vietnam. Currently, He is third year computer science student at SUTD, pursuing AI and Software engineering track. In this semester, he joined NUS, school of computing department as part of SUSEP program. \n\n\n\n\n'}]}, {'messages': [{'role': 'system', 'content': 'You are an assistant that answers all questions related to the candidate Tran Cong Nam Anh, referred to as his full name, or Louis Anh Tran, as his short name. The questions can

In [105]:
import json

with open('instances.jsonl','w') as f:
    for conversation in all_conversations:
        json.dump(conversation,f)
        f.write('\n')

In [106]:
client=openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [107]:
with open('instances.jsonl','rb') as f:
  response=client.files.create(file=f,purpose="fine-tune")

In [108]:
print(response)

FileObject(id='file-mlRYCINxK9Sazavpvaf3GXp5', bytes=13110, created_at=1713897000, filename='instances.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [109]:
print(response.id)

file-mlRYCINxK9Sazavpvaf3GXp5


In [110]:
file_id=response.id

In [112]:
response=client.fine_tuning.jobs.create(
  training_file=file_id, 
  model="gpt-3.5-turbo"
)

In [113]:
print(response)

FineTuningJob(id='ftjob-KR0SnvNnDYoHpJFFITufIuM2', created_at=1713897024, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-XQoG18cFDwdNAUfOwyUW2fPm', result_files=[], seed=1209080531, status='validating_files', trained_tokens=None, training_file='file-mlRYCINxK9Sazavpvaf3GXp5', validation_file=None, integrations=[], user_provided_suffix=None)


In [114]:
print(response.id)

ftjob-KR0SnvNnDYoHpJFFITufIuM2


In [115]:
job_id=response.id

In [137]:
client.fine_tuning.jobs.retrieve(job_id)


FineTuningJob(id='ftjob-KR0SnvNnDYoHpJFFITufIuM2', created_at=1713897024, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9HFIKpx5', finished_at=1713897395, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-XQoG18cFDwdNAUfOwyUW2fPm', result_files=['file-zuVIqNhKxZxpCtC8RBh1MRGc'], seed=1209080531, status='succeeded', trained_tokens=24450, training_file='file-mlRYCINxK9Sazavpvaf3GXp5', validation_file=None, integrations=[], user_provided_suffix=None)

In [138]:
tuned_model_id='ft:gpt-3.5-turbo-0125:personal::9HFIKpx5'

In [143]:
completion = client.chat.completions.create(
  model=tuned_model_id,
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What programming languages does he know?"}
  ]
)
print(completion.choices[0].message.content)

Louis is proficient in Java, Python, and has some experience with JavaScript and C.
